# Atividade 1



In [1]:
# Bibliotecas utilizadas
import secrets

import gurobipy as gp
from gurobipy import *

In [2]:
#
# Funções auxiliares
#

# Range inclusive
def inclrange(min, max):
    return range(min, max + 1)

# Inteiro aleatório entre min e max inclusive
def rand(min, max):
    return min + secrets.randbelow(max - min + 1)


### Limites das variáveis das instâncias

```
|J| = {100, 200, 300, 400, 500, 600, 700, 800, 900, 1000}
|F| ∈ [|J|, 2|J|]
|L| ∈ [5, 10]
|M| ∈ [5, 10]
|P| ∈ [5, 10]
D_j,p ∈ [10, 20]
r_m,p,l ∈ [1, 5]
R_m,f ∈ [800, 1000]
C_l,f ∈ [80, 100]
p_p,l,f ∈ [10, 100]
t_p,f,j ∈ [10, 20]
```

In [3]:
#
# Instâncias (dados utilizados, fixos enquanto o notebook está aberto)
#
# No futuro, imprimir esses valores e substituir a variável `instancias` para fixá-los para entrega da tarefa.
#

instancias = []

for len_J in [100 * x for x in inclrange(1, 10)]:
    J = [x for x in range(0, len_J)]  # conjunto de clientes
    # Número de...
    len_F = rand(len(J), 2 * len(J))
    F = [x for x in range(0, len_F)]  # fábricas
    
    len_L = rand(5, 10)  
    L = [x for x in range(0, len_L)]  # máquinas
    
    len_M = rand(5, 10)  
    M = [x for x in range(0, len_M)]  # matérias primas
    
    len_P = rand(5, 10)
    P = [x for x in range(0, len_P)]  # produtos
    
    D = []  # D_j,p: demanda de cliente j pelo produto p
    for j in J:
        D.append([])
        for p in P:
            D[j].append([])
            D[j][p] = rand(10, 20)
    
    r = []  # r_m,p,l: quantidade de matéria prima m para produzir produto p na máquina l
    for m in M:
        r.append([])
        for p in P:
            r[m].append([])
            for l in L:
                r[m][p].append([])
                r[m][p][l] = rand(1, 5)
                
    R = []  # R_m,f: quantidade de matéria prima m disponível na fábrica f
    for m in M:
        R.append([])
        for f in F:
            R[m].append([])
            R[m][f] = rand(800, 1000)
            
    C = []  # C_l,f: capacidade de produção da máquina l na fábrica f
    for l in L:
        C.append([])
        for f in F:
            C[l].append([])
            C[l][f] = rand(80, 100)
            
    cp = []  # p_p,l,f -> cp_p,l,f: custo de produção do produto p na máquina l na fábrica f
    for p in P:
        cp.append([])
        for l in L:
            cp[p].append([])
            for f in F:
                cp[p][l].append([])
                cp[p][l][f] = rand(10, 100)
    
    ct = []  # t_p,f,j -> ct_p,f,j: custo de transporte do produto p da fábrica f ao consumidor j
    for p in P:
        ct.append([])
        for f in F:
            ct[p].append([])
            for j in J:
                ct[p][f].append([])
                ct[p][f][j] = rand(10, 20)
    
    instancia = {
        'J': J,
        'F': F,
        'L': L,
        'M': M,
        'P': P,
        'D': D,
        'r': r,
        'R': R,
        'C': C,
        'cp': cp,
        'ct': ct
    }
    instancias.append(instancia)

In [4]:
#
# Executar solução
#

def solucionar(J, F, L, M, P, D, r, R, C, cp, ct):
    # Criação do modelo
    modelo = gp.Model("Atividade 1")

    # Variáveis de decisão
    tonelada_transportada = modelo.addVars(P, F, J, lb=0, vtype=GRB.INTEGER)
    tonelada_produzida = modelo.addVars(P, L, F, lb=0, vtype=GRB.INTEGER)
    custo = modelo.addVar(lb=0, vtype=GRB.INTEGER)
    
    # Função objetivo
    modelo.setObjective(custo, GRB.MINIMIZE)
    
    modelo.setParam('TimeLimit',1800)
    
    modelo.addConstr(custo >= quicksum(quicksum(quicksum(tonelada_produzida.sum(p,l,f) * cp[p][l][f] for p in P) for f in F) for l in L) +
                              quicksum(quicksum(quicksum(tonelada_transportada.sum(p,f,j) * ct[p][f][j] for p in P) for f in F) for j in J))
    
    
    for p in P:
        for j in J:
            modelo.addConstr(quicksum(tonelada_transportada.sum(p,f,j) for f in F) == D[j][p])
            
    for p in P:
        for f in F:
            modelo.addConstr(quicksum(tonelada_transportada.sum(p,f,j) for j in J) == quicksum(tonelada_produzida.sum(p,l,f) for l in L))
    
    for f in F:
        for m in M:
            modelo.addConstr(quicksum(quicksum(tonelada_produzida.sum(p,l,f) * r[m][p][l] for p in P) for l in L) <= R[m][f])
            
    for f in F:
        for l in L:
            modelo.addConstr(quicksum(tonelada_produzida.sum(p,l,f) for p in P) <= C[l][f])
    
    modelo.update()

    modelo.write('atividade1{}clientes.lp'.format(len(J)))

    modelo.optimize()
    
    if modelo.status == GRB.Status.OPTIMAL:
        print('Solução ótima:')
        print('Custo: {}'.format(modelo.objVal))

        for p in P:
            for l in L:
                for f in F:
                    if tonelada_produzida[p,l,f].X > 0.0:
                        print('Quantidade do produto {} produzido pela máquina {} na fábrica {} : {}'.format(p,l,f,tonelada_produzida[p,l,f].X))

        for p in P:
            for f in F:
                for j in J:
                    if tonelada_transportada[p,f,j].X > 0.0:
                        print('Quantidade do produto {} transportado da fábrica {} para o cliente {} : {}'.format(p,f,j,tonelada_transportada[p,f,j].X))

        print('Ótima solução:')
        print('Custo: {}'.format(modelo.objVal))
    elif modelo.status == GRB.Status.TIME_LIMIT:
        print('Não foi possível encontrar uma solução ótima. Tempo excedido.')
        print('Custo: {}'.format(modelo.objVal))

        for p in P:
            for l in L:
                for f in F:
                    if tonelada_produzida[p,l,f].X > 0.0:
                        print('Quantidade do produto {} produzido pela máquina {} na fábrica {} : {}'.format(p,l,f,tonelada_produzida[p,l,f].X))

        for p in P:
            for f in F:
                for j in J:
                    if tonelada_transportada[p,f,j].X > 0.0:
                        print('Quantidade do produto {} transportado da fábrica {} para o cliente {} : {}'.format(p,f,j,tonelada_transportada[p,f,j].X))

        print('Não foi possível encontrar uma solução ótima. Tempo excedido.')
        print('Solução: ')
        print('Custo: {}'.format(modelo.objVal))
    elif modelo.status == GRB.Status.INFEASIBLE:
        print('Não foi possível encontrar uma solução ótima. Status: Inviável.')
    else:
        print('Status do modelo: {}'.format(modelo.Status))

In [5]:
solucionar(**instancias[0])

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-17
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2653 rows, 68041 columns and 230041 nonzeros
Model fingerprint: 0x11b32f61
Variable types: 0 continuous, 68041 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 630258.00000
Presolve removed 1 rows and 1 columns
Presolve time: 0.30s
Presolved: 2652 rows, 68040 columns, 162000 nonzeros
Variable types: 0 continuous, 68040 integer (0 binary)
Found heuristic solution: objective 620083.00000

Root relaxation: objective 1.920144e+05, 4507 iterations, 0.31 seconds (0.11 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work

Quantidade do produto 1 transportado da fábrica 84 para o cliente 55 : 17.0
Quantidade do produto 1 transportado da fábrica 84 para o cliente 65 : 12.0
Quantidade do produto 1 transportado da fábrica 84 para o cliente 82 : 10.0
Quantidade do produto 1 transportado da fábrica 84 para o cliente 87 : 14.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 3 : 12.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 29 : 12.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 32 : 8.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 43 : 14.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 79 : 15.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 84 : 15.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 88 : 10.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 92 : 5.0
Quantidade do produto 1 transportado da fábrica 85 para o cliente 98 : 6.0
Quantidade do pr

Quantidade do produto 3 transportado da fábrica 56 para o cliente 64 : 14.0
Quantidade do produto 3 transportado da fábrica 56 para o cliente 71 : 12.0
Quantidade do produto 3 transportado da fábrica 56 para o cliente 83 : 15.0
Quantidade do produto 3 transportado da fábrica 56 para o cliente 89 : 10.0
Quantidade do produto 3 transportado da fábrica 64 para o cliente 22 : 12.0
Quantidade do produto 3 transportado da fábrica 64 para o cliente 29 : 11.0
Quantidade do produto 3 transportado da fábrica 64 para o cliente 72 : 13.0
Quantidade do produto 3 transportado da fábrica 64 para o cliente 88 : 12.0
Quantidade do produto 3 transportado da fábrica 64 para o cliente 98 : 15.0
Quantidade do produto 3 transportado da fábrica 64 para o cliente 99 : 15.0
Quantidade do produto 3 transportado da fábrica 65 para o cliente 10 : 16.0
Quantidade do produto 3 transportado da fábrica 65 para o cliente 15 : 15.0
Quantidade do produto 3 transportado da fábrica 65 para o cliente 31 : 19.0
Quantidade d

Quantidade do produto 5 transportado da fábrica 61 para o cliente 87 : 15.0
Quantidade do produto 5 transportado da fábrica 65 para o cliente 8 : 4.0
Quantidade do produto 5 transportado da fábrica 65 para o cliente 18 : 19.0
Quantidade do produto 5 transportado da fábrica 65 para o cliente 20 : 19.0
Quantidade do produto 5 transportado da fábrica 65 para o cliente 68 : 18.0
Quantidade do produto 5 transportado da fábrica 65 para o cliente 75 : 13.0
Quantidade do produto 5 transportado da fábrica 71 para o cliente 52 : 11.0
Quantidade do produto 5 transportado da fábrica 71 para o cliente 98 : 11.0
Quantidade do produto 5 transportado da fábrica 71 para o cliente 99 : 13.0
Quantidade do produto 5 transportado da fábrica 83 para o cliente 11 : 10.0
Quantidade do produto 5 transportado da fábrica 83 para o cliente 30 : 13.0
Quantidade do produto 5 transportado da fábrica 83 para o cliente 57 : 15.0
Quantidade do produto 5 transportado da fábrica 83 para o cliente 62 : 12.0
Quantidade do 

In [6]:
solucionar(**instancias[1])

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8261 rows, 551449 columns and 1747585 nonzeros
Model fingerprint: 0xa4d58da6
Variable types: 0 continuous, 551449 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 1713077.0000
Presolve removed 1 rows and 1 columns
Presolve time: 1.56s
Presolved: 8260 rows, 551448 columns, 1196136 nonzeros
Variable types: 0 continuous, 551448 integer (0 binary)
Found heuristic solution: objective 1679400.0000

Root relaxation: objective 4.857149e+05, 17084 iterations, 2.51 seconds (1.12 work units)
Total elapsed time = 5.04s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

Quantidade do produto 0 transportado da fábrica 140 para o cliente 143 : 15.0
Quantidade do produto 0 transportado da fábrica 158 para o cliente 30 : 16.0
Quantidade do produto 0 transportado da fábrica 158 para o cliente 42 : 15.0
Quantidade do produto 0 transportado da fábrica 158 para o cliente 94 : 12.0
Quantidade do produto 0 transportado da fábrica 158 para o cliente 171 : 15.0
Quantidade do produto 0 transportado da fábrica 158 para o cliente 184 : 10.0
Quantidade do produto 0 transportado da fábrica 158 para o cliente 186 : 20.0
Quantidade do produto 0 transportado da fábrica 163 para o cliente 29 : 16.0
Quantidade do produto 0 transportado da fábrica 163 para o cliente 191 : 3.0
Quantidade do produto 0 transportado da fábrica 169 para o cliente 1 : 9.0
Quantidade do produto 0 transportado da fábrica 169 para o cliente 6 : 18.0
Quantidade do produto 0 transportado da fábrica 169 para o cliente 7 : 13.0
Quantidade do produto 0 transportado da fábrica 169 para o cliente 61 : 15.0

Quantidade do produto 2 transportado da fábrica 0 para o cliente 1 : 8.0
Quantidade do produto 2 transportado da fábrica 0 para o cliente 16 : 13.0
Quantidade do produto 2 transportado da fábrica 0 para o cliente 60 : 11.0
Quantidade do produto 2 transportado da fábrica 0 para o cliente 88 : 13.0
Quantidade do produto 2 transportado da fábrica 0 para o cliente 132 : 15.0
Quantidade do produto 2 transportado da fábrica 0 para o cliente 193 : 18.0
Quantidade do produto 2 transportado da fábrica 0 para o cliente 197 : 13.0
Quantidade do produto 2 transportado da fábrica 4 para o cliente 46 : 11.0
Quantidade do produto 2 transportado da fábrica 4 para o cliente 54 : 11.0
Quantidade do produto 2 transportado da fábrica 4 para o cliente 59 : 12.0
Quantidade do produto 2 transportado da fábrica 4 para o cliente 63 : 8.0
Quantidade do produto 2 transportado da fábrica 4 para o cliente 93 : 10.0
Quantidade do produto 2 transportado da fábrica 4 para o cliente 104 : 12.0
Quantidade do produto 2 

Quantidade do produto 2 transportado da fábrica 214 para o cliente 1 : 3.0
Quantidade do produto 2 transportado da fábrica 214 para o cliente 30 : 16.0
Quantidade do produto 2 transportado da fábrica 214 para o cliente 73 : 17.0
Quantidade do produto 2 transportado da fábrica 214 para o cliente 116 : 4.0
Quantidade do produto 2 transportado da fábrica 214 para o cliente 127 : 12.0
Quantidade do produto 2 transportado da fábrica 214 para o cliente 131 : 16.0
Quantidade do produto 2 transportado da fábrica 214 para o cliente 146 : 18.0
Quantidade do produto 2 transportado da fábrica 218 para o cliente 0 : 11.0
Quantidade do produto 2 transportado da fábrica 218 para o cliente 8 : 14.0
Quantidade do produto 2 transportado da fábrica 218 para o cliente 40 : 6.0
Quantidade do produto 2 transportado da fábrica 218 para o cliente 77 : 14.0
Quantidade do produto 2 transportado da fábrica 218 para o cliente 117 : 18.0
Quantidade do produto 2 transportado da fábrica 218 para o cliente 130 : 14.0

Quantidade do produto 3 transportado da fábrica 182 para o cliente 82 : 19.0
Quantidade do produto 3 transportado da fábrica 182 para o cliente 99 : 10.0
Quantidade do produto 3 transportado da fábrica 182 para o cliente 150 : 14.0
Quantidade do produto 3 transportado da fábrica 182 para o cliente 160 : 3.0
Quantidade do produto 3 transportado da fábrica 185 para o cliente 2 : 16.0
Quantidade do produto 3 transportado da fábrica 185 para o cliente 3 : 2.0
Quantidade do produto 3 transportado da fábrica 185 para o cliente 58 : 14.0
Quantidade do produto 3 transportado da fábrica 185 para o cliente 101 : 13.0
Quantidade do produto 3 transportado da fábrica 185 para o cliente 135 : 12.0
Quantidade do produto 3 transportado da fábrica 185 para o cliente 181 : 11.0
Quantidade do produto 3 transportado da fábrica 185 para o cliente 188 : 20.0
Quantidade do produto 3 transportado da fábrica 219 para o cliente 27 : 10.0
Quantidade do produto 3 transportado da fábrica 219 para o cliente 62 : 3.

Quantidade do produto 4 transportado da fábrica 169 para o cliente 122 : 14.0
Quantidade do produto 4 transportado da fábrica 169 para o cliente 173 : 14.0
Quantidade do produto 4 transportado da fábrica 169 para o cliente 177 : 20.0
Quantidade do produto 4 transportado da fábrica 170 para o cliente 23 : 10.0
Quantidade do produto 4 transportado da fábrica 170 para o cliente 28 : 19.0
Quantidade do produto 4 transportado da fábrica 170 para o cliente 46 : 17.0
Quantidade do produto 4 transportado da fábrica 170 para o cliente 96 : 19.0
Quantidade do produto 4 transportado da fábrica 170 para o cliente 125 : 3.0
Quantidade do produto 4 transportado da fábrica 170 para o cliente 139 : 16.0
Quantidade do produto 4 transportado da fábrica 170 para o cliente 141 : 12.0
Quantidade do produto 4 transportado da fábrica 176 para o cliente 64 : 14.0
Quantidade do produto 4 transportado da fábrica 176 para o cliente 154 : 20.0
Quantidade do produto 4 transportado da fábrica 177 para o cliente 7 :

Quantidade do produto 5 transportado da fábrica 112 para o cliente 6 : 16.0
Quantidade do produto 5 transportado da fábrica 112 para o cliente 33 : 17.0
Quantidade do produto 5 transportado da fábrica 112 para o cliente 64 : 16.0
Quantidade do produto 5 transportado da fábrica 112 para o cliente 105 : 3.0
Quantidade do produto 5 transportado da fábrica 112 para o cliente 114 : 17.0
Quantidade do produto 5 transportado da fábrica 112 para o cliente 125 : 13.0
Quantidade do produto 5 transportado da fábrica 120 para o cliente 25 : 12.0
Quantidade do produto 5 transportado da fábrica 120 para o cliente 58 : 10.0
Quantidade do produto 5 transportado da fábrica 120 para o cliente 109 : 10.0
Quantidade do produto 5 transportado da fábrica 120 para o cliente 139 : 18.0
Quantidade do produto 5 transportado da fábrica 120 para o cliente 142 : 1.0
Quantidade do produto 5 transportado da fábrica 120 para o cliente 182 : 11.0
Quantidade do produto 5 transportado da fábrica 120 para o cliente 183 :

Quantidade do produto 6 transportado da fábrica 53 para o cliente 21 : 13.0
Quantidade do produto 6 transportado da fábrica 53 para o cliente 24 : 14.0
Quantidade do produto 6 transportado da fábrica 53 para o cliente 57 : 12.0
Quantidade do produto 6 transportado da fábrica 53 para o cliente 106 : 14.0
Quantidade do produto 6 transportado da fábrica 53 para o cliente 140 : 18.0
Quantidade do produto 6 transportado da fábrica 53 para o cliente 149 : 6.0
Quantidade do produto 6 transportado da fábrica 53 para o cliente 191 : 4.0
Quantidade do produto 6 transportado da fábrica 81 para o cliente 56 : 16.0
Quantidade do produto 6 transportado da fábrica 81 para o cliente 66 : 18.0
Quantidade do produto 6 transportado da fábrica 81 para o cliente 118 : 10.0
Quantidade do produto 6 transportado da fábrica 81 para o cliente 133 : 2.0
Quantidade do produto 6 transportado da fábrica 81 para o cliente 171 : 18.0
Quantidade do produto 6 transportado da fábrica 81 para o cliente 188 : 19.0
Quantid

Quantidade do produto 7 transportado da fábrica 61 para o cliente 29 : 4.0
Quantidade do produto 7 transportado da fábrica 61 para o cliente 35 : 10.0
Quantidade do produto 7 transportado da fábrica 61 para o cliente 81 : 14.0
Quantidade do produto 7 transportado da fábrica 61 para o cliente 149 : 16.0
Quantidade do produto 7 transportado da fábrica 61 para o cliente 175 : 19.0
Quantidade do produto 7 transportado da fábrica 68 para o cliente 28 : 10.0
Quantidade do produto 7 transportado da fábrica 68 para o cliente 31 : 15.0
Quantidade do produto 7 transportado da fábrica 68 para o cliente 34 : 11.0
Quantidade do produto 7 transportado da fábrica 68 para o cliente 55 : 20.0
Quantidade do produto 7 transportado da fábrica 68 para o cliente 114 : 14.0
Quantidade do produto 7 transportado da fábrica 68 para o cliente 141 : 20.0
Quantidade do produto 7 transportado da fábrica 68 para o cliente 143 : 9.0
Quantidade do produto 7 transportado da fábrica 75 para o cliente 48 : 19.0
Quantidad

In [7]:
solucionar(**instancias[2])

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 14530 rows, 1679941 columns and 5204521 nonzeros
Model fingerprint: 0xdeea1141
Variable types: 0 continuous, 1679941 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 3180855.0000
Presolve removed 1 rows and 1 columns
Presolve time: 4.44s
Presolved: 14529 rows, 1679940 columns, 3524580 nonzeros
Variable types: 0 continuous, 1679940 integer (0 binary)
Found heuristic solution: objective 3120612.0000

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.3715600e+05   1.149594e+04   3.440497e+10     14s
 

Quantidade do produto 1 produzido pela máquina 2 na fábrica 423 : 17.0
Quantidade do produto 1 produzido pela máquina 2 na fábrica 508 : 89.0
Quantidade do produto 1 produzido pela máquina 2 na fábrica 517 : 92.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 28 : 97.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 86 : 15.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 176 : 81.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 235 : 84.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 238 : 90.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 302 : 100.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 306 : 87.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 440 : 96.0
Quantidade do produto 1 produzido pela máquina 3 na fábrica 458 : 93.0
Quantidade do produto 1 produzido pela máquina 4 na fábrica 101 : 80.0
Quantidade do produto 1 produzido pela máquina 4 na fábrica 121 : 97.0
Quantid

Quantidade do produto 0 transportado da fábrica 7 para o cliente 36 : 12.0
Quantidade do produto 0 transportado da fábrica 7 para o cliente 90 : 9.0
Quantidade do produto 0 transportado da fábrica 7 para o cliente 159 : 16.0
Quantidade do produto 0 transportado da fábrica 7 para o cliente 173 : 16.0
Quantidade do produto 0 transportado da fábrica 7 para o cliente 203 : 12.0
Quantidade do produto 0 transportado da fábrica 7 para o cliente 216 : 13.0
Quantidade do produto 0 transportado da fábrica 7 para o cliente 256 : 12.0
Quantidade do produto 0 transportado da fábrica 7 para o cliente 290 : 3.0
Quantidade do produto 0 transportado da fábrica 32 para o cliente 164 : 2.0
Quantidade do produto 0 transportado da fábrica 49 para o cliente 223 : 11.0
Quantidade do produto 0 transportado da fábrica 49 para o cliente 253 : 10.0
Quantidade do produto 0 transportado da fábrica 49 para o cliente 288 : 1.0
Quantidade do produto 0 transportado da fábrica 50 para o cliente 43 : 8.0
Quantidade do p

Quantidade do produto 0 transportado da fábrica 278 para o cliente 83 : 15.0
Quantidade do produto 0 transportado da fábrica 278 para o cliente 131 : 3.0
Quantidade do produto 0 transportado da fábrica 278 para o cliente 157 : 20.0
Quantidade do produto 0 transportado da fábrica 278 para o cliente 186 : 14.0
Quantidade do produto 0 transportado da fábrica 278 para o cliente 210 : 6.0
Quantidade do produto 0 transportado da fábrica 278 para o cliente 214 : 15.0
Quantidade do produto 0 transportado da fábrica 278 para o cliente 297 : 10.0
Quantidade do produto 0 transportado da fábrica 281 para o cliente 12 : 16.0
Quantidade do produto 0 transportado da fábrica 281 para o cliente 93 : 12.0
Quantidade do produto 0 transportado da fábrica 281 para o cliente 117 : 20.0
Quantidade do produto 0 transportado da fábrica 281 para o cliente 140 : 6.0
Quantidade do produto 0 transportado da fábrica 281 para o cliente 146 : 15.0
Quantidade do produto 0 transportado da fábrica 281 para o cliente 199

Quantidade do produto 0 transportado da fábrica 546 para o cliente 7 : 14.0
Quantidade do produto 0 transportado da fábrica 546 para o cliente 11 : 11.0
Quantidade do produto 0 transportado da fábrica 546 para o cliente 55 : 16.0
Quantidade do produto 0 transportado da fábrica 546 para o cliente 114 : 15.0
Quantidade do produto 0 transportado da fábrica 546 para o cliente 119 : 10.0
Quantidade do produto 0 transportado da fábrica 546 para o cliente 137 : 2.0
Quantidade do produto 0 transportado da fábrica 546 para o cliente 239 : 20.0
Quantidade do produto 0 transportado da fábrica 548 para o cliente 24 : 10.0
Quantidade do produto 0 transportado da fábrica 548 para o cliente 76 : 13.0
Quantidade do produto 0 transportado da fábrica 548 para o cliente 90 : 10.0
Quantidade do produto 0 transportado da fábrica 548 para o cliente 191 : 17.0
Quantidade do produto 0 transportado da fábrica 548 para o cliente 236 : 15.0
Quantidade do produto 0 transportado da fábrica 548 para o cliente 285 :

...

Quantidade do produto 7 transportado da fábrica 331 para o cliente 82 : 14.0
Quantidade do produto 7 transportado da fábrica 331 para o cliente 98 : 12.0
Quantidade do produto 7 transportado da fábrica 331 para o cliente 114 : 14.0
Quantidade do produto 7 transportado da fábrica 331 para o cliente 157 : 11.0
Quantidade do produto 7 transportado da fábrica 331 para o cliente 194 : 5.0
Quantidade do produto 7 transportado da fábrica 331 para o cliente 226 : 14.0
Quantidade do produto 7 transportado da fábrica 331 para o cliente 295 : 20.0
Quantidade do produto 7 transportado da fábrica 342 para o cliente 43 : 20.0
Quantidade do produto 7 transportado da fábrica 342 para o cliente 110 : 14.0
Quantidade do produto 7 transportado da fábrica 342 para o cliente 181 : 20.0
Quantidade do produto 7 transportado da fábrica 342 para o cliente 249 : 19.0
Quantidade do produto 7 transportado da fábrica 342 para o cliente 268 : 15.0
Quantidade do produto 7 transportado da fábrica 342 para o cliente 2

Quantidade do produto 7 transportado da fábrica 516 para o cliente 3 : 16.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 61 : 20.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 66 : 10.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 116 : 19.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 124 : 17.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 141 : 15.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 153 : 19.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 158 : 14.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 206 : 13.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 207 : 2.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 211 : 18.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 254 : 13.0
Quantidade do produto 7 transportado da fábrica 516 para o cliente 29

Quantidade do produto 8 transportado da fábrica 217 para o cliente 161 : 20.0
Quantidade do produto 8 transportado da fábrica 217 para o cliente 210 : 10.0
Quantidade do produto 8 transportado da fábrica 217 para o cliente 284 : 19.0
Quantidade do produto 8 transportado da fábrica 242 para o cliente 17 : 15.0
Quantidade do produto 8 transportado da fábrica 242 para o cliente 33 : 6.0
Quantidade do produto 8 transportado da fábrica 242 para o cliente 90 : 19.0
Quantidade do produto 8 transportado da fábrica 242 para o cliente 104 : 20.0
Quantidade do produto 8 transportado da fábrica 242 para o cliente 174 : 12.0
Quantidade do produto 8 transportado da fábrica 242 para o cliente 205 : 10.0
Quantidade do produto 8 transportado da fábrica 242 para o cliente 290 : 7.0
Quantidade do produto 8 transportado da fábrica 250 para o cliente 3 : 13.0
Quantidade do produto 8 transportado da fábrica 250 para o cliente 8 : 10.0
Quantidade do produto 8 transportado da fábrica 250 para o cliente 11 : 7

Quantidade do produto 8 transportado da fábrica 473 para o cliente 44 : 14.0
Quantidade do produto 8 transportado da fábrica 473 para o cliente 76 : 17.0
Quantidade do produto 8 transportado da fábrica 473 para o cliente 118 : 19.0
Quantidade do produto 8 transportado da fábrica 473 para o cliente 132 : 15.0
Quantidade do produto 8 transportado da fábrica 473 para o cliente 142 : 19.0
Quantidade do produto 8 transportado da fábrica 473 para o cliente 155 : 11.0
Quantidade do produto 8 transportado da fábrica 473 para o cliente 268 : 1.0
Quantidade do produto 8 transportado da fábrica 488 para o cliente 229 : 8.0
Quantidade do produto 8 transportado da fábrica 527 para o cliente 11 : 10.0
Quantidade do produto 8 transportado da fábrica 527 para o cliente 53 : 18.0
Quantidade do produto 8 transportado da fábrica 527 para o cliente 89 : 19.0
Quantidade do produto 8 transportado da fábrica 527 para o cliente 152 : 16.0
Quantidade do produto 8 transportado da fábrica 527 para o cliente 163 

Quantidade do produto 9 transportado da fábrica 176 para o cliente 36 : 11.0
Quantidade do produto 9 transportado da fábrica 176 para o cliente 126 : 16.0
Quantidade do produto 9 transportado da fábrica 176 para o cliente 148 : 17.0
Quantidade do produto 9 transportado da fábrica 176 para o cliente 166 : 19.0
Quantidade do produto 9 transportado da fábrica 176 para o cliente 186 : 10.0
Quantidade do produto 9 transportado da fábrica 176 para o cliente 218 : 2.0
Quantidade do produto 9 transportado da fábrica 176 para o cliente 253 : 8.0
Quantidade do produto 9 transportado da fábrica 178 para o cliente 63 : 5.0
Quantidade do produto 9 transportado da fábrica 178 para o cliente 89 : 17.0
Quantidade do produto 9 transportado da fábrica 178 para o cliente 199 : 10.0
Quantidade do produto 9 transportado da fábrica 178 para o cliente 233 : 13.0
Quantidade do produto 9 transportado da fábrica 178 para o cliente 251 : 10.0
Quantidade do produto 9 transportado da fábrica 178 para o cliente 281

In [8]:
solucionar(**instancias[3])

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20569 rows, 2185697 columns and 6989953 nonzeros
Model fingerprint: 0x482b3d6c
Variable types: 0 continuous, 2185697 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 3412145.0000
Presolve removed 1 rows and 1 columns (presolve time = 5s) ...
Presolve removed 1 rows and 1 columns
Presolve time: 6.38s
Presolved: 20568 rows, 2185696 columns, 4804256 nonzeros
Variable types: 0 continuous, 2185696 integer (0 binary)
Found heuristic solution: objective 3343707.0000

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Elapsed ordering time = 5s
Ordering time: 8.48s

Quantidade do produto 1 produzido pela máquina 0 na fábrica 370 : 86.0
Quantidade do produto 1 produzido pela máquina 0 na fábrica 443 : 95.0
Quantidade do produto 1 produzido pela máquina 0 na fábrica 538 : 89.0
Quantidade do produto 1 produzido pela máquina 1 na fábrica 42 : 10.0
Quantidade do produto 1 produzido pela máquina 1 na fábrica 110 : 87.0
Quantidade do produto 1 produzido pela máquina 1 na fábrica 160 : 100.0
Quantidade do produto 1 produzido pela máquina 1 na fábrica 234 : 95.0
Quantidade do produto 1 produzido pela máquina 1 na fábrica 363 : 92.0
Quantidade do produto 1 produzido pela máquina 1 na fábrica 597 : 88.0
Quantidade do produto 1 produzido pela máquina 1 na fábrica 647 : 91.0
Quantidade do produto 1 produzido pela máquina 2 na fábrica 51 : 76.0
Quantidade do produto 1 produzido pela máquina 2 na fábrica 271 : 78.0
Quantidade do produto 1 produzido pela máquina 2 na fábrica 321 : 85.0
Quantidade do produto 1 produzido pela máquina 2 na fábrica 399 : 88.0
Quantid

Quantidade do produto 0 transportado da fábrica 33 para o cliente 14 : 8.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 34 : 9.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 82 : 4.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 218 : 13.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 231 : 15.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 264 : 4.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 339 : 6.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 365 : 14.0
Quantidade do produto 0 transportado da fábrica 33 para o cliente 369 : 11.0
Quantidade do produto 0 transportado da fábrica 38 para o cliente 187 : 14.0
Quantidade do produto 0 transportado da fábrica 38 para o cliente 227 : 18.0
Quantidade do produto 0 transportado da fábrica 38 para o cliente 244 : 9.0
Quantidade do produto 0 transportado da fábrica 38 para o cliente 310 : 14.0
Quantida

Quantidade do produto 0 transportado da fábrica 429 para o cliente 18 : 20.0
Quantidade do produto 0 transportado da fábrica 429 para o cliente 140 : 15.0
Quantidade do produto 0 transportado da fábrica 429 para o cliente 154 : 19.0
Quantidade do produto 0 transportado da fábrica 429 para o cliente 272 : 15.0
Quantidade do produto 0 transportado da fábrica 429 para o cliente 371 : 12.0
Quantidade do produto 0 transportado da fábrica 429 para o cliente 386 : 2.0
Quantidade do produto 0 transportado da fábrica 430 para o cliente 139 : 19.0
Quantidade do produto 0 transportado da fábrica 430 para o cliente 185 : 12.0
Quantidade do produto 0 transportado da fábrica 430 para o cliente 193 : 13.0
Quantidade do produto 0 transportado da fábrica 430 para o cliente 212 : 14.0
Quantidade do produto 0 transportado da fábrica 430 para o cliente 303 : 19.0
Quantidade do produto 0 transportado da fábrica 430 para o cliente 339 : 14.0
Quantidade do produto 0 transportado da fábrica 432 para o cliente

Quantidade do produto 1 transportado da fábrica 1 para o cliente 332 : 10.0
Quantidade do produto 1 transportado da fábrica 1 para o cliente 399 : 10.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 77 : 15.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 142 : 13.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 227 : 17.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 266 : 4.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 276 : 5.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 334 : 16.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 360 : 15.0
Quantidade do produto 1 transportado da fábrica 11 para o cliente 384 : 9.0
Quantidade do produto 1 transportado da fábrica 33 para o cliente 167 : 6.0
Quantidade do produto 1 transportado da fábrica 33 para o cliente 291 : 14.0
Quantidade do produto 1 transportado da fábrica 33 para o cliente 349 : 11.0
Quanti

...

Quantidade do produto 5 transportado da fábrica 573 para o cliente 200 : 19.0
Quantidade do produto 5 transportado da fábrica 573 para o cliente 208 : 12.0
Quantidade do produto 5 transportado da fábrica 573 para o cliente 229 : 19.0
Quantidade do produto 5 transportado da fábrica 573 para o cliente 280 : 8.0
Quantidade do produto 5 transportado da fábrica 573 para o cliente 317 : 15.0
Quantidade do produto 5 transportado da fábrica 573 para o cliente 336 : 14.0
Quantidade do produto 5 transportado da fábrica 576 para o cliente 136 : 14.0
Quantidade do produto 5 transportado da fábrica 576 para o cliente 138 : 16.0
Quantidade do produto 5 transportado da fábrica 576 para o cliente 139 : 14.0
Quantidade do produto 5 transportado da fábrica 576 para o cliente 180 : 17.0
Quantidade do produto 5 transportado da fábrica 576 para o cliente 241 : 17.0
Quantidade do produto 5 transportado da fábrica 576 para o cliente 246 : 7.0
Quantidade do produto 5 transportado da fábrica 576 para o cliente

Quantidade do produto 6 transportado da fábrica 150 para o cliente 13 : 12.0
Quantidade do produto 6 transportado da fábrica 150 para o cliente 79 : 10.0
Quantidade do produto 6 transportado da fábrica 150 para o cliente 95 : 12.0
Quantidade do produto 6 transportado da fábrica 150 para o cliente 109 : 11.0
Quantidade do produto 6 transportado da fábrica 150 para o cliente 253 : 17.0
Quantidade do produto 6 transportado da fábrica 150 para o cliente 320 : 12.0
Quantidade do produto 6 transportado da fábrica 150 para o cliente 369 : 13.0
Quantidade do produto 6 transportado da fábrica 161 para o cliente 39 : 11.0
Quantidade do produto 6 transportado da fábrica 161 para o cliente 110 : 17.0
Quantidade do produto 6 transportado da fábrica 161 para o cliente 181 : 19.0
Quantidade do produto 6 transportado da fábrica 161 para o cliente 311 : 3.0
Quantidade do produto 6 transportado da fábrica 161 para o cliente 350 : 16.0
Quantidade do produto 6 transportado da fábrica 161 para o cliente 39

Quantidade do produto 6 transportado da fábrica 570 para o cliente 27 : 17.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 83 : 15.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 90 : 17.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 144 : 20.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 254 : 16.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 279 : 12.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 287 : 20.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 294 : 14.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 300 : 11.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 311 : 12.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 319 : 13.0
Quantidade do produto 6 transportado da fábrica 570 para o cliente 368 : 11.0
Quantidade do produto 6 transportado da fábrica 572 para o cliente 

Quantidade do produto 7 transportado da fábrica 107 para o cliente 55 : 10.0
Quantidade do produto 7 transportado da fábrica 107 para o cliente 83 : 15.0
Quantidade do produto 7 transportado da fábrica 107 para o cliente 225 : 14.0
Quantidade do produto 7 transportado da fábrica 107 para o cliente 240 : 16.0
Quantidade do produto 7 transportado da fábrica 107 para o cliente 259 : 18.0
Quantidade do produto 7 transportado da fábrica 107 para o cliente 362 : 10.0
Quantidade do produto 7 transportado da fábrica 107 para o cliente 370 : 12.0
Quantidade do produto 7 transportado da fábrica 110 para o cliente 21 : 9.0
Quantidade do produto 7 transportado da fábrica 110 para o cliente 56 : 18.0
Quantidade do produto 7 transportado da fábrica 110 para o cliente 104 : 13.0
Quantidade do produto 7 transportado da fábrica 110 para o cliente 186 : 12.0
Quantidade do produto 7 transportado da fábrica 110 para o cliente 236 : 7.0
Quantidade do produto 7 transportado da fábrica 110 para o cliente 243

Quantidade do produto 7 transportado da fábrica 458 para o cliente 17 : 17.0
Quantidade do produto 7 transportado da fábrica 458 para o cliente 28 : 10.0
Quantidade do produto 7 transportado da fábrica 458 para o cliente 45 : 12.0
Quantidade do produto 7 transportado da fábrica 458 para o cliente 106 : 10.0
Quantidade do produto 7 transportado da fábrica 458 para o cliente 134 : 12.0
Quantidade do produto 7 transportado da fábrica 458 para o cliente 216 : 10.0
Quantidade do produto 7 transportado da fábrica 458 para o cliente 335 : 17.0
Quantidade do produto 7 transportado da fábrica 458 para o cliente 366 : 12.0
Quantidade do produto 7 transportado da fábrica 468 para o cliente 113 : 16.0
Quantidade do produto 7 transportado da fábrica 468 para o cliente 143 : 20.0
Quantidade do produto 7 transportado da fábrica 468 para o cliente 144 : 13.0
Quantidade do produto 7 transportado da fábrica 468 para o cliente 153 : 13.0
Quantidade do produto 7 transportado da fábrica 468 para o cliente 

In [ ]:
solucionar(**instancias[4])

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 23263 rows, 4017641 columns and 12386401 nonzeros
Model fingerprint: 0x36bb307d
Variable types: 0 continuous, 4017641 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 5208486.0000
Presolve removed 1 rows and 1 columns (presolve time = 5s) ...
Presolve removed 1 rows and 1 columns (presolve time = 10s) ...
Presolve removed 1 rows and 1 columns
Presolve time: 13.89s
Presolved: 23262 rows, 4017640 columns, 8368760 nonzeros
Variable types: 0 continuous, 4017640 integer (0 binary)
Found heuristic solution: objective 5111012.0000

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ro

In [ ]:
solucionar(**instancias[5])

In [ ]:
solucionar(**instancias[6])

In [ ]:
solucionar(**instancias[7])

In [ ]:
solucionar(**instancias[8])

In [ ]:
solucionar(**instancias[9])

In [ ]:
solucionar(**instancias[10])